## 1 Augmentation. Architecture Improvements and Optimization process

In [1]:
import itertools
import random
import logging

from pathlib import Path
from dataclasses import dataclass
from collections import defaultdict

from typing import Optional, Union, Callable, Set, List, Dict

import numpy as np
import pandas as pd
from tqdm.auto import tqdm
import editdistance
import cv2

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import torch.nn.utils.rnn as utils_rnn

from torchvision.transforms import transforms

import albumentations as A
from albumentations.augmentations import functional as AF
from albumentations.pytorch import ToTensorV2

In [2]:
DATA_DIR = Path("./data")
EXP_DIR = Path("./exp/advanced_methods_model")
EXP_DIR.mkdir(exist_ok=True, parents=True)

In [3]:
def set_seed():
    random.seed(321)
    torch.manual_seed(321)
    torch.cuda.manual_seed(321)
    np.random.seed(321)

In [4]:
def setup_logger(exp_dir: Optional[Union[str, Path]] = None):
    logger = logging.getLogger()
    logger.setLevel(logging.INFO)
    logger.handlers.clear()

    c_handler = logging.StreamHandler()
    c_format = logging.Formatter("%(name)s:%(lineno)d - %(levelname)s - %(message)s")
    c_handler.setFormatter(c_format)
    logger.addHandler(c_handler)

    if exp_dir is not None:
        exp_dir = Path(exp_dir)
        f_handler = logging.FileHandler(exp_dir / "learning.log", mode="w")
        f_format = logging.Formatter("%(asctime)s - %(name)s:%(lineno)d - %(levelname)s - %(message)s")
        f_handler.setFormatter(f_format)
        logger.addHandler(f_handler)

In [5]:
set_seed()
setup_logger(EXP_DIR)

## 1.1 Loading Dataset

In [6]:
dataset_info = pd.read_csv(DATA_DIR / "mapper.csv")

In [7]:
dataset_info.head()

,Unnamed: 0,new_name,old_name,train,public,private
0,0,15_8_11.jpg,165_8_11.jpg,0,0,1
1,1,32_2_1.jpg,380_2_1.jpg,1,0,0
2,2,0_17_10.jpg,5_17_10.jpg,1,0,0
3,3,21_10_5.jpg,183_10_5.jpg,0,0,1
4,4,38_23_7.jpg,416_23_7.jpg,1,0,0


In [8]:
excluded_files = {"20_16_0.jpg", "41_10_1.jpg", "47_20_5.jpg", "313_12_9.jpg"}
train_val_uttids = set(map(lambda x: x.rsplit(".")[0], dataset_info[
    (dataset_info.train == 1) & (~dataset_info.old_name.isin(excluded_files))].new_name.values.tolist()))

In [9]:
len(train_val_uttids)

6192

In [10]:
def get_train_val_uttids(train_val_uttids):
    uttids_blocks = defaultdict(list)
    for uttid in train_val_uttids:
        series, page, line = map(int, uttid.split("_"))
        uttids_blocks[(series, page)].append(line)
    keys = sorted(uttids_blocks.keys())
    num_train_keys = len(keys) * 14 // 15
    np.random.seed(111)
    np.random.shuffle(keys)
    train_keys = keys[:num_train_keys]
    val_keys = keys[num_train_keys:]
    train_uttids = []
    val_uttids = []
    for series, page in train_keys:
        for line in sorted(uttids_blocks[(series, page)]):
            uttid = f"{series}_{page}_{line}"
            train_uttids.append(uttid)
    for series, page in val_keys:
        for line in sorted(uttids_blocks[(series, page)]):
            uttid = f"{series}_{page}_{line}"
            val_uttids.append(uttid)
    return set(train_uttids), set(val_uttids)

In [11]:
train_uttids, val_uttids = get_train_val_uttids(train_val_uttids)

In [12]:
len(train_uttids), len(val_uttids)

(5699, 493)

In [13]:
chars = {" ", "+", "[", "]", "⊕",
         "0", "1", "2", "3", "4", "5", "6", "7", "8", "9", 
         "b", "c", "d", "e", "f", "g", "h", "i", "m", "n", "p", "r", "s", "t", 
         "а", "б", "в", "г", "д", "е", "ж", "з", "и", "й", "к", "л", "м", "н", 
         "о", "п", "р", "с", "т", "у", "ф", "х", "ц", "ч", "ш", "щ", "ъ", "ы", "ь", "э", "ю", "я", "ѣ"
        }

In [14]:
len(chars)

62

In [15]:
class TextEncoder:
    def __init__(self, allowed_chars: Set[str]):
        self.allowed_chars = allowed_chars
        self.id2char = ["<blank>"] + sorted(allowed_chars)
        self.char2id = dict(zip(self.id2char, range(len(self.id2char))))

    def encode(self, text: str):
        return [self.char2id[c] for c in text if c in self.allowed_chars]

    def decode(self, sequence: List[int]):
        return "".join(self.id2char[i] for i in sequence)

    def decode_ctc(self, sequence: List[int]):
        return self.decode([i for i, _ in itertools.groupby(sequence) if i != 0])

In [16]:
text_encoder = TextEncoder(chars)

In [17]:
### Dataset

In [86]:
def clean_text(text: str) -> str:
    replacements = [
        ("і", "i"),
        ("c", "с"),
        ("a", "а"),
        ("⊗", "⊕"),
        ("lll", " "),
        ("–", " "),
        (")", "]"),
        ("(", "["),
        ("|", " "),
        ("×", "+"),
        ("k", "к"),
        ("ǂ", "+"),
    ]
    text = text.strip()
    for char_in, char_out in replacements:
        text = text.replace(char_in, char_out)
    text = " ".join(text.strip().split())  # remove additional spaces
    return text

In [19]:
def process_image(img, target_height=128):
    height, width, _ = img.shape
    if height > width:
        img = np.rot90(img)
        height, width = width, height

    new_height = target_height
    new_width = int(round(width * new_height / height))
    img = cv2.resize(img, (new_width, new_height))  # sic!
    height, width, _ = img.shape

    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB) 

    if height < target_height:
        top_zeros_height = (target_height - height) // 2
        bottom_zeros_height = target_height - height - top_zeros_height
        top_zeros = np.full((top_zeros_height, width, 3), 255, dtype=img.dtype)
        bottom_zeros = np.full((bottom_zeros_height, width, 3), 255, dtype=img.dtype)
        img = np.concatenate((top_zeros, img, bottom_zeros))
        height, width, _ = img.shape

    return img

In [20]:
@dataclass
class OcrDataItem:
    key: str
    img: torch.Tensor
    text: str
    encoded_text: torch.Tensor
    encoded_text_len: int


@dataclass
class OcrDataBatch:
    keys: List[str]
    texts: List[str]
    images: torch.Tensor
    encoded_texts: torch.Tensor
    image_lengths: torch.Tensor
    text_lengths: torch.Tensor

In [21]:
class DigitalPeterDataset(Dataset):
    def __init__(self,
                 base_dir: Union[Path, str],
                 uttids: Set[str],
                 encoder: TextEncoder,
                 img_height=128,
                 image_len_divisible_by=4,
                 training=False,
                 sort=False):
        super().__init__()
        base_dir = Path(base_dir)
        self.trans_dir = base_dir / "words"
        self.image_dir = base_dir / "images"
        self.images = []
        self.texts = []
        self.encoded_texts = []
        self.keys = []
        self.encoder = encoder
        self.training = training
        self.img_height = img_height

        def pad_pil_image(img, **params):
            height, width, _ = img.shape
            tail = width % image_len_divisible_by
            if tail == 0:
                return img
            # args for pad: left, top, right, bottom
            # return functional.pad(img, (0, 0, image_len_divisible_by - tail, 0), padding_mode="edge")
            return AF.pad(img, min_height=height, min_width=width + image_len_divisible_by - tail)

        def random_stretch_image(img, **params):
            # return functional.resize(img, (img.height, int(img.width * random.uniform(0.95, 1.05))))
            height, width, _ = img.shape
            return AF.resize(img, height, int(width * random.uniform(0.95, 1.05)))

        self.train_transforms = A.Compose([
            A.Rotate(limit=4),
            A.Lambda(random_stretch_image, p=0.5),
            A.Lambda(pad_pil_image),
            A.GridDistortion(p=0.5),
            A.ToFloat(max_value=255),
            ToTensorV2(),
        ])
        self.eval_transforms = transforms.ToTensor()
        self.image_len_divisible_by = image_len_divisible_by
        log = logging.getLogger(__name__)
        
        for uttid in tqdm(sorted(uttids)):
            imagepath = self.image_dir / f"{uttid}.jpg"
            textpath = self.trans_dir / f"{uttid}.txt"
            with open(textpath, "r", encoding="utf-8") as f:
                text = f.read().strip()
            encoded_text = self.encoder.encode(clean_text(text))

            self.keys.append(uttid)
            self.texts.append(text)
            self.encoded_texts.append(torch.LongTensor(encoded_text))
            img = cv2.imread(f"{imagepath}")
            img = process_image(img, self.img_height)
            width = img.shape[1]
            if width % image_len_divisible_by != 0:
                right_add = np.full([img.shape[0], image_len_divisible_by - width % image_len_divisible_by, 3], 255,
                                    dtype=img.dtype)
                img = np.concatenate((img, right_add), axis=1)

            self.images.append(img)  # HxWxC -> CxHxW later
        self._indices = list(range(len(self.keys)))

    def __getitem__(self, index: int) -> OcrDataItem:
        index = self._indices[index]
        img = self.images[index]
        if self.training:
            img = self.train_transforms(image=img)["image"]
        else:
            img = self.eval_transforms(img)
        return OcrDataItem(self.keys[index], img, self.texts[index], self.encoded_texts[index],
                           self.encoded_texts[index].shape[-1])

    def __len__(self):
        return len(self.keys)

In [22]:
train_dataset = DigitalPeterDataset(DATA_DIR, train_uttids, text_encoder, training=True)

  0%|          | 0/5699 [00:00<?, ?it/s]

In [23]:
val_dataset = DigitalPeterDataset(DATA_DIR, val_uttids, text_encoder, training=False)

  0%|          | 0/493 [00:00<?, ?it/s]

In [24]:
def collate_fn(items: List[OcrDataItem]):
    items.sort(key=lambda item: -item.img.shape[-1])  # sort by image width
    image_lengths = torch.LongTensor([item.img.shape[-1] for item in items])
    images = utils_rnn.pad_sequence(
        [item.img.transpose(0, -1) for item in items],
        batch_first=False).permute(1, 3, 2, 0)  # CxHxW -> WxHxC -> WxBxHxC -> BxCxHxW
    encoded_texts = utils_rnn.pad_sequence([item.encoded_text for item in items], batch_first=True)
    texts = [item.text for item in items]
    keys = [item.key for item in items]
    text_lengths = torch.LongTensor([item.encoded_text_len for item in items])
    return OcrDataBatch(keys, texts, images, encoded_texts, image_lengths, text_lengths)

In [25]:
train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True, collate_fn=collate_fn)
val_loader = DataLoader(val_dataset, batch_size=4, shuffle=False, collate_fn=collate_fn)

## 1.2 Defining the Model

In [26]:
class LambdaModule(nn.Module):
    def __init__(self, f: Callable):
        super().__init__()
        self.f = f

    def forward(self, *args):
        return self.f(*args)

In [27]:
class ConvExtractor(nn.Module):
    def __init__(self, norm=nn.BatchNorm2d):
        super().__init__()
        # input: Bx3x128xL
        left_context = 19
        right_context = 19 + 4
        self.reduction_fn = lambda x: x // 4
        self.encoder = nn.Sequential(*[
            nn.ReplicationPad2d([left_context, right_context, 0, 0]),
            norm(3),
            nn.Conv2d(3, 64, kernel_size=(3, 3), padding=[1, 0]),  # L - 2
            nn.ReLU(),
            norm(64),
            nn.MaxPool2d(kernel_size=(4, 2), stride=2),  # / 2
            nn.Conv2d(64, 128, kernel_size=(3, 3), padding=[1, 0]),  # -2
            nn.ReLU(),
            norm(128),
            nn.MaxPool2d(kernel_size=(4, 2), stride=2),  # /2
            nn.Conv2d(128, 256, (3, 3), padding=[1, 0]),  # -2
            nn.ReLU(),
            norm(256),
            nn.Conv2d(256, 256, (3, 3), padding=[1, 0]),  # -2
            nn.ReLU(),
            norm(256),
            nn.ZeroPad2d([0, 0, 2, 1]),  # same padding for maxpool2d
            nn.MaxPool2d(kernel_size=(4, 1), padding=0),  # pool_4
            nn.Conv2d(256, 512, (3, 3), padding=[1, 0]),  # -2
            nn.ReLU(),
            norm(512),
            nn.Conv2d(512, 512, (3, 3), padding=[1, 0]),  # -2
            nn.ReLU(),
            norm(512),
            nn.ZeroPad2d([0, 0, 1, 2]),  # same padding for maxpool2d
            nn.MaxPool2d(kernel_size=(4, 1), padding=0),
            nn.Conv2d(512, 512, (2, 2)),  # 512x1x255 CxHxW # -1
            LambdaModule(lambda x: x.squeeze(dim=2).permute(2, 0, 1)),  # LxBxC
        ])

    def forward(self, images, image_lengths):
        output = self.encoder(images)  # LxBxC, L//4
        return output, self.reduction_fn(image_lengths)  # LxBxC, B

In [28]:
class RNNEncoder(nn.Module):
    def __init__(self, dropout=0.2, num_layers=2, rnn_type="GRU", hidden_size=256, input_size=512):
        super().__init__()
        self.rnn_dropout = nn.Dropout(dropout)
        rnn_type = getattr(nn, rnn_type)
        self.num_layers = num_layers
        assert num_layers >= 1
        self.rnn = rnn_type(input_size=input_size, hidden_size=hidden_size, bidirectional=True, dropout=dropout,
                            batch_first=False,
                            num_layers=num_layers)

    def forward(self, sequences, sequence_lengths):
        output = self.rnn_dropout(sequences)
        output = utils_rnn.pack_padded_sequence(output, sequence_lengths.cpu(), batch_first=False)
        output = self.rnn(output)[0]
        output = utils_rnn.pad_packed_sequence(output)[0]
        return output, sequence_lengths  # LxBxC, B

In [29]:
class ConvGRU2x368(nn.Module):
    def __init__(self, num_outputs: int):
        super().__init__()
        self.conv_extractor = ConvExtractor()
        self.rnn_encoder = RNNEncoder(dropout=0.2, rnn_type="GRU", num_layers=2, hidden_size=368, input_size=512)
        self.fc = nn.Linear(368*2, num_outputs)
    
    def forward(self, images, image_lengths):
        output, output_lengths = self.conv_extractor(images, image_lengths)
        output = F.relu(output)
        output, output_lenghts = self.rnn_encoder(output, output_lengths)
        output = F.relu(output)
        output = self.fc(output)
        return output, output_lenghts

In [30]:
class SequentialSequential(nn.Sequential):
    def forward(self, *inputs):
        for module in self:
            inputs = module(*inputs)
        return inputs

In [31]:
class SequentialLinear(nn.Module):
    def __init__(self, num_inputs, num_outputs, pre_activation=True):
        super().__init__()
        modules = []
        if pre_activation:
            modules.append(nn.ReLU())
        modules.append(nn.Linear(num_inputs, num_outputs))
        self.model = nn.Sequential(*modules)

    def forward(self, sequences, sequence_lengths):
        return self.model(sequences), sequence_lengths

In [32]:
num_outputs = len(chars) + 1

In [33]:
set_seed()

In [34]:
model = SequentialSequential(*[
        ConvExtractor(),
        LambdaModule(lambda seq, seq_len: (F.relu(seq), seq_len)),
        RNNEncoder(dropout=0.2, rnn_type="GRU", num_layers=2, hidden_size=368, input_size=512),
        SequentialLinear(368 * 2, num_outputs, pre_activation=True)
    ])
model = model.cuda()

## 1.3 Training

In [35]:
num_epochs = 48
warmup_epochs = 4

In [36]:
optimizer = optim.SGD(model.parameters(), lr=1e-7, momentum=0.9, weight_decay=1e-2)

In [37]:
# using cyclic learning rate schedule

In [38]:
reduce_lr = optim.lr_scheduler.CyclicLR(
                optimizer, base_lr=1e-7, max_lr=1e-2,
                cycle_momentum=True,
                step_size_up=len(train_loader) * warmup_epochs,
                step_size_down=len(train_loader) * (num_epochs - warmup_epochs))

In [39]:
def calc_metrics(utt2hyp: Dict[str, str], utt2ref: Dict[str, str]):
    error_chars = 0
    total_chars = 0
    error_words = 0
    total_words = 0
    error_sentences = 0
    log = logging.getLogger(__name__)

    for i, (uttid, hyp) in enumerate(utt2hyp.items()):
        ref = utt2ref[uttid]
        total_chars += len(ref)
        total_words += len(ref.split())
        error_chars += editdistance.eval(hyp, ref)
        error_words += editdistance.eval(hyp.split(), ref.split())
        if ref != hyp:
            error_sentences += 1
        if i < 20:
            log.info(f"{uttid} ref: {ref}")
            log.info(f"{uttid} hyp: {hyp}")
    cer = error_chars / total_chars
    wer = error_words / total_words
    num_sentences = len(utt2hyp)
    sentence_accuracy = (num_sentences - error_sentences) / num_sentences
    log.info(f"CER: {cer * 100:.3f}%, WER: {wer * 100:.3f}%, SACC: {sentence_accuracy * 100:.3f}%")
    return cer, wer

In [40]:
criterion = nn.CTCLoss(blank=0, reduction="none")

In [41]:
def train_model():
    model.train()
    tmp_loss = 0.0
    tmp_loss_num = 0
    ocr_data_batch: OcrDataBatch
    for batch_idx, ocr_data_batch in enumerate(tqdm(train_loader)):
        images = ocr_data_batch.images.cuda()
        image_lengths = ocr_data_batch.image_lengths.cuda()
        encoded_texts = ocr_data_batch.encoded_texts.cuda().to(torch.int32)  # for ctc
        text_lengths = ocr_data_batch.text_lengths.cuda().to(torch.int32)  # for ctc

        optimizer.zero_grad()
        logits, logits_lengths = model(images, image_lengths)
        log_logits = F.log_softmax(logits, dim=-1)
        loss = criterion(log_logits, encoded_texts, logits_lengths, text_lengths).mean()
        if not loss.isnan():
            loss.backward()
            optimizer.step()

            tmp_loss_num += 1
            tmp_loss += loss.item()
        if (batch_idx + 1) % 100 == 0 and tmp_loss_num != 0:
            logging.info(f"loss: {tmp_loss / tmp_loss_num:.5f}")
            tmp_loss_num = 0
            tmp_loss = 0.0
        reduce_lr.step()

In [42]:
def val_model_greedy(loader):
    model.eval()
    num_items = 0
    loss_accum = 0.0

    utt2hyp: Dict[str, str] = dict()
    utt2ref: Dict[str, str] = dict()
    with torch.no_grad():
        ocr_data_batch: OcrDataBatch
        for batch_idx, ocr_data_batch in enumerate(tqdm(loader)):
            images = ocr_data_batch.images.cuda()
            image_lengths = ocr_data_batch.image_lengths.cuda()
            encoded_texts = ocr_data_batch.encoded_texts.cuda().to(torch.int32)  # for ctc
            text_lengths = ocr_data_batch.text_lengths.cuda().to(torch.int32)  # for ctc
            batch_size = images.shape[0]

            logits, logits_lengths = model(images, image_lengths)
            log_logits = F.log_softmax(logits, dim=-1)
            loss = criterion(log_logits, encoded_texts, logits_lengths, text_lengths)
            loss_accum += loss.sum().item()
            num_items += batch_size

            cpu_log_logits = log_logits.transpose(0, 1).detach().cpu()
            for i in range(batch_size):
                key = ocr_data_batch.keys[i]
                cur_logits = cpu_log_logits[i, :logits_lengths[i]].detach()
                utt2ref[key] = ocr_data_batch.texts[i]
                
            labels = logits.argmax(dim=-1).detach().cpu().numpy().transpose(1, 0)
        

            for i, ref in enumerate(ocr_data_batch.texts):
                key = ocr_data_batch.keys[i]
                hyp_len = logits_lengths[i].item()
                hyp = text_encoder.decode_ctc(labels[i].tolist()[:hyp_len]).strip()
                utt2hyp[key] = hyp

    loss_accum /= num_items
    log.info(f"loss: {loss_accum:.5f}")
    _ = calc_metrics(utt2hyp, utt2ref)
    return loss_accum

In [43]:
exp_dir = EXP_DIR

In [44]:
log = logging.getLogger("trainscript")
best_loss = val_model_greedy(val_loader)
torch.save(model.state_dict(), exp_dir / "model_best.pt")
try:
    for i_epoch in range(num_epochs):
        log.info("=" * 50)
        log.info(f"epoch: {i_epoch + 1}")
        train_model()
        cur_loss = val_model_greedy(val_loader)
        if best_loss < cur_loss:
            log.info(f"not improved {best_loss:.5f} < {cur_loss:.5f}")
            torch.save(model.state_dict(), exp_dir / "model_last.pt")
        else:
            log.info(f"improved {best_loss:.5f} -> {cur_loss:.5f}")
            best_loss = cur_loss
            torch.save(model.state_dict(), exp_dir / "model_best.pt")
        if i_epoch + 5 >= num_epochs:
            torch.save(model.state_dict(), exp_dir / f"model_ep{i_epoch + 1}.pt")
except KeyboardInterrupt:
    log.warning("training interruped")

  0%|          | 0/124 [00:00<?, ?it/s]

trainscript:39 - INFO - loss: 786.11769
__main__:18 - INFO - 0_13_10 ref: ца куды конюнктуры поворотятца
__main__:19 - INFO - 0_13_10 hyp: з
__main__:18 - INFO - 0_13_1 ref: катеринушка друг мой здравъствуй
__main__:19 - INFO - 0_13_1 hyp: з
__main__:18 - INFO - 0_13_11 ref: iз лагоу при подеме
__main__:19 - INFO - 0_13_11 hyp: з
__main__:18 - INFO - 0_13_0 ref: +
__main__:19 - INFO - 0_13_0 hyp: з
__main__:18 - INFO - 0_13_15 ref: вам молитца а нам трудитца
__main__:19 - INFO - 0_13_15 hyp: з
__main__:18 - INFO - 0_13_14 ref: + надобно понеже время пришло
__main__:19 - INFO - 0_13_14 hyp: з
__main__:18 - INFO - 0_13_13 ref: въ 2 д декабря 1712
__main__:19 - INFO - 0_13_13 hyp: з
__main__:18 - INFO - 0_13_12 ref: петръ
__main__:19 - INFO - 0_13_12 hyp: з
__main__:18 - INFO - 0_13_4 ref: ты чтоб взят вас сюды i о том тепер отло
__main__:19 - INFO - 0_13_4 hyp: з
__main__:18 - INFO - 0_13_2 ref: благодарствую за пълате которое обно
__main__:19 - INFO - 0_13_2 hyp: з
__main__:18 - INFO - 

  0%|          | 0/1425 [00:00<?, ?it/s]

root:23 - INFO - loss: 158.82869
root:23 - INFO - loss: 92.98377
root:23 - INFO - loss: 95.82618
root:23 - INFO - loss: 90.91395
root:23 - INFO - loss: 92.50870
root:23 - INFO - loss: 92.68061
root:23 - INFO - loss: 90.26840
root:23 - INFO - loss: 95.30631
root:23 - INFO - loss: 96.29228
root:23 - INFO - loss: 93.60099
root:23 - INFO - loss: 91.70375
root:23 - INFO - loss: 89.87415
root:23 - INFO - loss: 91.33670
root:23 - INFO - loss: 86.51281


  0%|          | 0/124 [00:00<?, ?it/s]

...

trainscript:39 - INFO - loss: 5.02194
__main__:18 - INFO - 0_13_10 ref: ца куды конюнктуры поворотятца
__main__:19 - INFO - 0_13_10 hyp: ца куды конюнктуры поворотятца
__main__:18 - INFO - 0_13_1 ref: катеринушка друг мой здравъствуй
__main__:19 - INFO - 0_13_1 hyp: катеринушка друг мой здравъствуй
__main__:18 - INFO - 0_13_11 ref: iз лагоу при подеме
__main__:19 - INFO - 0_13_11 hyp: iз лаго у при подеме
__main__:18 - INFO - 0_13_0 ref: +
__main__:19 - INFO - 0_13_0 hyp: +
__main__:18 - INFO - 0_13_15 ref: вам молитца а нам трудитца
__main__:19 - INFO - 0_13_15 hyp: вам молитца а нам трудитца
__main__:18 - INFO - 0_13_14 ref: + надобно понеже время пришло
__main__:19 - INFO - 0_13_14 hyp: + надобно понеже время пришло
__main__:18 - INFO - 0_13_13 ref: въ 2 д декабря 1712
__main__:19 - INFO - 0_13_13 hyp: въ 2д декабря 1712
__main__:18 - INFO - 0_13_12 ref: петръ
__main__:19 - INFO - 0_13_12 hyp: петръ
__main__:18 - INFO - 0_13_4 ref: ты чтоб взят вас сюды i о том тепер отло
__main__:1

  0%|          | 0/1425 [00:00<?, ?it/s]

root:23 - INFO - loss: 4.88499
root:23 - INFO - loss: 5.34488
root:23 - INFO - loss: 4.45707
root:23 - INFO - loss: 5.57391
root:23 - INFO - loss: 4.35399
root:23 - INFO - loss: 4.69663
root:23 - INFO - loss: 4.67475
root:23 - INFO - loss: 4.42407
root:23 - INFO - loss: 4.75385
root:23 - INFO - loss: 4.72955
root:23 - INFO - loss: 5.39797
root:23 - INFO - loss: 4.82506
root:23 - INFO - loss: 4.82417
root:23 - INFO - loss: 4.92741


  0%|          | 0/124 [00:00<?, ?it/s]

trainscript:39 - INFO - loss: 4.73043
__main__:18 - INFO - 0_13_10 ref: ца куды конюнктуры поворотятца
__main__:19 - INFO - 0_13_10 hyp: ца куды конюнктуры поворотятца
__main__:18 - INFO - 0_13_1 ref: катеринушка друг мой здравъствуй
__main__:19 - INFO - 0_13_1 hyp: катеринушка друг гой здравъствуй
__main__:18 - INFO - 0_13_11 ref: iз лагоу при подеме
__main__:19 - INFO - 0_13_11 hyp: iз лаго у при подеме
__main__:18 - INFO - 0_13_0 ref: +
__main__:19 - INFO - 0_13_0 hyp: +
__main__:18 - INFO - 0_13_15 ref: вам молитца а нам трудитца
__main__:19 - INFO - 0_13_15 hyp: вам молитца а нам трудитца
__main__:18 - INFO - 0_13_14 ref: + надобно понеже время пришло
__main__:19 - INFO - 0_13_14 hyp: + надобно понеже время пришло
__main__:18 - INFO - 0_13_13 ref: въ 2 д декабря 1712
__main__:19 - INFO - 0_13_13 hyp: въ 2 д декабря 1712
__main__:18 - INFO - 0_13_12 ref: петръ
__main__:19 - INFO - 0_13_12 hyp: петръ
__main__:18 - INFO - 0_13_4 ref: ты чтоб взят вас сюды i о том тепер отло
__main__:

In [45]:
model.load_state_dict(torch.load(exp_dir / "model_best.pt"))
val_model_greedy(val_loader)

  0%|          | 0/124 [00:00<?, ?it/s]

trainscript:39 - INFO - loss: 4.73043
__main__:18 - INFO - 0_13_10 ref: ца куды конюнктуры поворотятца
__main__:19 - INFO - 0_13_10 hyp: ца куды конюнктуры поворотятца
__main__:18 - INFO - 0_13_1 ref: катеринушка друг мой здравъствуй
__main__:19 - INFO - 0_13_1 hyp: катеринушка друг гой здравъствуй
__main__:18 - INFO - 0_13_11 ref: iз лагоу при подеме
__main__:19 - INFO - 0_13_11 hyp: iз лаго у при подеме
__main__:18 - INFO - 0_13_0 ref: +
__main__:19 - INFO - 0_13_0 hyp: +
__main__:18 - INFO - 0_13_15 ref: вам молитца а нам трудитца
__main__:19 - INFO - 0_13_15 hyp: вам молитца а нам трудитца
__main__:18 - INFO - 0_13_14 ref: + надобно понеже время пришло
__main__:19 - INFO - 0_13_14 hyp: + надобно понеже время пришло
__main__:18 - INFO - 0_13_13 ref: въ 2 д декабря 1712
__main__:19 - INFO - 0_13_13 hyp: въ 2 д декабря 1712
__main__:18 - INFO - 0_13_12 ref: петръ
__main__:19 - INFO - 0_13_12 hyp: петръ
__main__:18 - INFO - 0_13_4 ref: ты чтоб взят вас сюды i о том тепер отло
__main__:

4.730425802737656

# 2 Beam Search Decoding with Language Model

## 2.1 Installing tools: SRILM with LBFGS

You can install manually SRILM with LBFGS, use Kaldi tools or Kaldi scripts: 
* https://github.com/kaldi-asr/kaldi/blob/master/tools/extras/install_liblbfgs.sh
* https://github.com/kaldi-asr/kaldi/blob/master/tools/extras/install_srilm.sh

To install SRILM, download sources from the official web site: http://www.speech.sri.com/projects/srilm/ (use VPN, if you live in Russia). 

If the site is unavailable, you can use unofficial mirror on Github: https://github.com/BitSpeech/SRILM

In [46]:
!mkdir -p ./tools
%cd ./tools

!wget https://github.com/downloads/chokkan/liblbfgs/liblbfgs-1.10.tar.gz
!tar -xzf liblbfgs-1.10.tar.gz
%cd liblbfgs-1.10
!./configure --prefix=`pwd`
!make -j 6
!make -i install
%cd ../

!git clone --recursive git@github.com:BitSpeech/SRILM.git
%cd SRILM
!mv Makefile _Makefile
!echo "SRILM=$(pwd)" > Makefile
!cat _Makefile >> Makefile

!echo HAVE_LIBLBFGS=1 >> common/Makefile.machine.i686-m64
!grep ADDITIONAL_INCLUDES common/Makefile.machine.i686-m64 | \
    sed 's|$| -I$(SRILM)/../liblbfgs-1.10/include|' \
    >> common/Makefile.machine.i686-m64

!grep ADDITIONAL_LDFLAGS common/Makefile.machine.i686-m64 | \
    sed 's|$| -L$(SRILM)/../liblbfgs-1.10/lib/ -Wl,-rpath -Wl,$(SRILM)/../liblbfgs-1.10/lib/|' \
    >> common/Makefile.machine.i686-m64

!make -j 6
%cd ../../

## 2.2 Estimate char-level Language Model

### Data from Digital Peter dataset

In [79]:
def convert_for_lm(text: str) -> str:
    text = clean_text(text)
    text = " ".join(text.replace(" ", "$"))
    text = text.replace("[", "P").replace("]", "Q")
    return text

In [80]:
train_text_lm = []
for uttid in sorted(train_uttids):
    with open(DATA_DIR / f"words/{uttid}.txt", "r", encoding="utf-8") as f:
        text = f.read().strip()
        train_text_lm.append(convert_for_lm(text))
        
train_text_lm = sorted(set(train_text_lm))

with open(EXP_DIR / "train_texts_for_lm.txt", "w", encoding="utf-8") as f:
    print("\n".join(train_text_lm), file=f)

In [83]:
!./tools/SRILM/bin/i686-m64/ngram-count \
    -text $EXP_DIR/train_texts_for_lm.txt \
    -order 6 -wbdiscount -interpolate -maxent -maxent-convert-to-arpa \
    -lm $EXP_DIR/lm_train_o6_wbimaxent.gz

Starting fitting...
Starting OWL-BFGS with c1=3.10825e-06, sigma2=965172, max_iters=1000
  No of NaNs in logZs: 0, No infs: 0
  dual is 4.12713
  regularized dual is 4.12713
  norm of gradient =0.215316
  norm of regularized gradient =0.215316
  No of NaNs in logZs: 0, No infs: 0
  dual is 3.9208
  regularized dual is 3.9208
  norm of gradient =0.196219
  norm of regularized gradient =0.196218
Iteration 1
  No of NaNs in logZs: 0, No infs: 0
  dual is 4.16589
  regularized dual is 4.16597
  norm of gradient =0.634241
  norm of regularized gradient =0.634246
  No of NaNs in logZs: 0, No infs: 0
  dual is 3.3269
  regularized dual is 3.32693
  norm of gradient =0.191187
  norm of regularized gradient =0.191188
Iteration 2
  No of NaNs in logZs: 0, No infs: 0
  dual is 3.11398
  regularized dual is 3.11402
  norm of gradient =0.0714989
  norm of regularized gradient =0.0714946
Iteration 3
  No of NaNs in logZs: 0, No infs: 0
  dual is 3.00185
  regularized dual is 3.00191
  norm of gradie

### Data from GramEval2020  
We use historical texts of the 17 century from GramEval 2020 https://github.com/dialogue-evaluation/GramEval2020

In [84]:
GRAM_EVAL_PATH = Path("./external_data/grameval_17_century.txt")

In [87]:
def clean_text_grameval(text, chars_to_remove):
    replacements = [
        ("x", "х"),
        ("ѳ", "ф"),
        ("ѡ", "о"),
        ("і", "i"),
        ("ѯ", "кс"),
    ]
    strange_chars = {"ѱ", "є", "v"}
    for char_in, char_out in replacements:
        text = text.replace(char_in, char_out)
    for char in chars_to_remove:
        text = text.replace(char, " ")
    text = " ".join(word for word in text.split() if len(set(word).intersection(strange_chars)) == 0)
    return text

In [88]:
texts = set()
with open(GRAM_EVAL_PATH, "r", encoding="utf-8") as f:
    for line in f:
        texts.add(" ".join(line.strip().split()))
char_geval = set("".join(texts))
chars_to_remove = char_geval - chars - {"ѱ", "є", "v", "x", "ѳ", "ѡ", "і", "ѯ"}
texts = set(clean_text_grameval(text, chars_to_remove) for text in texts) - {""}
char_geval = set("".join(texts))
with open(f"{EXP_DIR / 'grameval_17_century_cleaned.txt'}", "w", encoding="utf-8") as f_texts, \
        open(f"{EXP_DIR / 'grameval_17_century_cleaned_for_lm.txt'}", "w", encoding="utf-8") as f_texts_lm:
    for text in texts:
        print(text, file=f_texts)
        print(" ".join(text.replace(" ", "$")), file=f_texts_lm)

In [89]:
!./tools/SRILM/bin/i686-m64/ngram-count \
    -text $EXP_DIR/grameval_17_century_cleaned_for_lm.txt \
    -order 6 -wbdiscount -interpolate -maxent -maxent-convert-to-arpa \
    -lm $EXP_DIR/lm_geval17_o6_wbimaxent.gz

Starting fitting...
Starting OWL-BFGS with c1=3.34769e-08, sigma2=8.96139e+07, max_iters=1000
  No of NaNs in logZs: 0, No infs: 0
  dual is 3.89182
  regularized dual is 3.89182
  norm of gradient =0.214306
  norm of regularized gradient =0.214306
  No of NaNs in logZs: 0, No infs: 0
  dual is 3.68833
  regularized dual is 3.68833
  norm of gradient =0.191374
  norm of regularized gradient =0.191374
Iteration 1
  No of NaNs in logZs: 0, No infs: 0
  dual is 3.61178
  regularized dual is 3.61178
  norm of gradient =0.524625
  norm of regularized gradient =0.524625
Iteration 2
  No of NaNs in logZs: 0, No infs: 0
  dual is 2.94542
  regularized dual is 2.94542
  norm of gradient =0.11126
  norm of regularized gradient =0.11126
Iteration 3
  No of NaNs in logZs: 0, No infs: 0
  dual is 2.89002
  regularized dual is 2.89002
  norm of gradient =0.0687474
  norm of regularized gradient =0.0687473
Iteration 4
  No of NaNs in logZs: 0, No infs: 0
  dual is 2.78598
  regularized dual is 2.7859

In [90]:
!./tools/SRILM/bin/i686-m64/ngram \
    -order 6 -lm $EXP_DIR/lm_train_o6_wbimaxent.gz -mix-lm $EXP_DIR/lm_geval17_o6_wbimaxent.gz -lambda 0.8 \
    -write-lm $EXP_DIR/lm_train_geval17_06_wbimaxent_0.8.gz

## 2.3 Decode with Beam Search

In [54]:
# !pip install git+https://github.com/parlance/ctcdecode.git

In [73]:
from ctcdecode import CTCBeamDecoder

In [74]:
phones_list = text_encoder.id2char.copy()

phones_list[phones_list.index(" ")] = "$"
phones_list[phones_list.index("[")] = "P"
phones_list[phones_list.index("]")] = "Q"

In [75]:
train_lm_decoder = CTCBeamDecoder(
        phones_list,
        model_path=f"{EXP_DIR / 'lm_train_o6_wbimaxent.gz'}",
        alpha=0.8,
        beta=2.0,
        cutoff_top_n=40,
        cutoff_prob=1.0,
        beam_width=100,
        num_processes=12,
        blank_id=0,
        log_probs_input=True
    )

In [76]:
def val_model_with_decoder(loader, lm_decoder):
    model.eval()
    num_items = 0
    loss_accum = 0.0

    utt2hyp: Dict[str, str] = dict()
    utt2ref: Dict[str, str] = dict()
    with torch.no_grad():
        ocr_data_batch: OcrDataBatch
        for batch_idx, ocr_data_batch in enumerate(tqdm(loader)):
            images = ocr_data_batch.images.cuda()
            image_lengths = ocr_data_batch.image_lengths.cuda()
            encoded_texts = ocr_data_batch.encoded_texts.cuda().to(torch.int32)  # for ctc
            text_lengths = ocr_data_batch.text_lengths.cuda().to(torch.int32)  # for ctc
            batch_size = images.shape[0]

            logits, logits_lengths = model(images, image_lengths)
            log_logits = F.log_softmax(logits, dim=-1)
            loss = criterion(log_logits, encoded_texts, logits_lengths, text_lengths)
            loss_accum += loss.sum().item()
            num_items += batch_size

            # save logits
            cpu_log_logits = log_logits.transpose(0, 1).detach().cpu()
            for i in range(batch_size):
                key = ocr_data_batch.keys[i]
                cur_logits = cpu_log_logits[i, :logits_lengths[i]].detach()
                utt2ref[key] = ocr_data_batch.texts[i]
                
            beam_results, beam_scores, timesteps, out_lens = lm_decoder.decode(
                log_logits.transpose(0, 1).detach(),
                seq_lens=logits_lengths)

            for i, ref in enumerate(ocr_data_batch.texts):
                key = ocr_data_batch.keys[i]
                hyp_len = out_lens[i][0]
                hyp_encoded = beam_results[i, 0, :hyp_len]
                hyp = text_encoder.decode(hyp_encoded.numpy().tolist()).strip()
                utt2hyp[key] = hyp

    loss_accum /= num_items
    log.info(f"loss: {loss_accum:.5f}")
    _ = calc_metrics(utt2hyp, utt2ref)
    return loss_accum

In [77]:
val_model_with_decoder(val_loader, train_lm_decoder)

  0%|          | 0/124 [00:00<?, ?it/s]

trainscript:42 - INFO - loss: 4.73043
__main__:18 - INFO - 0_13_10 ref: ца куды конюнктуры поворотятца
__main__:19 - INFO - 0_13_10 hyp: ца куды конюнктуры поворотятца
__main__:18 - INFO - 0_13_1 ref: катеринушка друг мой здравъствуй
__main__:19 - INFO - 0_13_1 hyp: катеринушка друг мой здравъствуй
__main__:18 - INFO - 0_13_11 ref: iз лагоу при подеме
__main__:19 - INFO - 0_13_11 hyp: iз лаго у при под еме
__main__:18 - INFO - 0_13_0 ref: +
__main__:19 - INFO - 0_13_0 hyp: +
__main__:18 - INFO - 0_13_15 ref: вам молитца а нам трудитца
__main__:19 - INFO - 0_13_15 hyp: вам молитца а нам трудитца
__main__:18 - INFO - 0_13_14 ref: + надобно понеже время пришло
__main__:19 - INFO - 0_13_14 hyp: + надобно понеже время пришло
__main__:18 - INFO - 0_13_13 ref: въ 2 д декабря 1712
__main__:19 - INFO - 0_13_13 hyp: въ 2 д декабря 1712
__main__:18 - INFO - 0_13_12 ref: петръ
__main__:19 - INFO - 0_13_12 hyp: петръ
__main__:18 - INFO - 0_13_4 ref: ты чтоб взят вас сюды i о том тепер отло
__main__

4.730425802737656

In [91]:
train_geval17_lm_decoder = CTCBeamDecoder(
        phones_list,
        model_path=f"{EXP_DIR / 'lm_train_geval17_06_wbimaxent_0.8.gz'}",
        alpha=0.8,
        beta=2.0,
        cutoff_top_n=40,
        cutoff_prob=1.0,
        beam_width=100,
        num_processes=12,
        blank_id=0,
        log_probs_input=True
    )

In [92]:
val_model_with_decoder(val_loader, train_geval17_lm_decoder)

  0%|          | 0/124 [00:00<?, ?it/s]

trainscript:42 - INFO - loss: 4.73043
__main__:18 - INFO - 0_13_10 ref: ца куды конюнктуры поворотятца
__main__:19 - INFO - 0_13_10 hyp: ца куды конюнктуры поворотятца
__main__:18 - INFO - 0_13_1 ref: катеринушка друг мой здравъствуй
__main__:19 - INFO - 0_13_1 hyp: катеринушка друг мой здравъствуй
__main__:18 - INFO - 0_13_11 ref: iз лагоу при подеме
__main__:19 - INFO - 0_13_11 hyp: iз лаго у при под еме
__main__:18 - INFO - 0_13_0 ref: +
__main__:19 - INFO - 0_13_0 hyp: +
__main__:18 - INFO - 0_13_15 ref: вам молитца а нам трудитца
__main__:19 - INFO - 0_13_15 hyp: вам молитца а нам трудитца
__main__:18 - INFO - 0_13_14 ref: + надобно понеже время пришло
__main__:19 - INFO - 0_13_14 hyp: + надобно понеже время пришло
__main__:18 - INFO - 0_13_13 ref: въ 2 д декабря 1712
__main__:19 - INFO - 0_13_13 hyp: въ 2 д декабря 1712
__main__:18 - INFO - 0_13_12 ref: петръ
__main__:19 - INFO - 0_13_12 hyp: петръ
__main__:18 - INFO - 0_13_4 ref: ты чтоб взят вас сюды i о том тепер отло
__main__

4.730425802737656

### Decoder on private test set

In [62]:
test_uttids = set(map(lambda x: x.rsplit(".")[0], 
                      dataset_info[dataset_info.private == 1].new_name.values.tolist()))

In [63]:
test_dataset = DigitalPeterDataset(DATA_DIR, test_uttids, text_encoder, training=False)

  0%|          | 0/1930 [00:00<?, ?it/s]

In [64]:
test_loader = DataLoader(test_dataset, batch_size=4, shuffle=False, collate_fn=collate_fn)

In [67]:
val_model_greedy(test_loader)

  0%|          | 0/483 [00:00<?, ?it/s]

trainscript:39 - INFO - loss: 5.51383
__main__:18 - INFO - 0_14_12 ref: съ юшковым поклонис от меня кнеiме да
__main__:19 - INFO - 0_14_12 hyp: съ юшковъм поклонис отменя к не iмеда
__main__:18 - INFO - 0_14_13 ref: ниловичевой
__main__:19 - INFO - 0_14_13 hyp: ниловивей
__main__:18 - INFO - 0_14_11 ref: p s благодарствую за пърисылку
__main__:19 - INFO - 0_14_11 hyp: p s благода авую за пърисылко
__main__:18 - INFO - 0_14_10 ref: iз гистроу въ 12 д декабря 1712
__main__:19 - INFO - 0_14_10 hyp: iз гистро у въ 12 д декабря 1712
__main__:18 - INFO - 0_18_13 ref: заполочным до элбинга а оттол отпусти iх з бата
__main__:19 - INFO - 0_18_13 hyp: за полочным до элбинга а оттол отпусти iх збата
__main__:18 - INFO - 0_18_12 ref: празником (+ також возми с собою i афицероф
__main__:19 - INFO - 0_18_12 hyp: празником [ також возми с обою i афицероф
__main__:18 - INFO - 0_18_11 ref: p s поздравъляю вам сим торжественным
__main__:19 - INFO - 0_18_11 hyp: p s поздравляю вам сим туржественным
__mai

5.513829678327926

In [78]:
val_model_with_decoder(test_loader, train_lm_decoder)

  0%|          | 0/483 [00:00<?, ?it/s]

trainscript:42 - INFO - loss: 5.51383
__main__:18 - INFO - 0_14_12 ref: съ юшковым поклонис от меня кнеiме да
__main__:19 - INFO - 0_14_12 hyp: съ юшковым поклонис от меня к не iмеда
__main__:18 - INFO - 0_14_13 ref: ниловичевой
__main__:19 - INFO - 0_14_13 hyp: нилови вой
__main__:18 - INFO - 0_14_11 ref: p s благодарствую за пърисылку
__main__:19 - INFO - 0_14_11 hyp: p s благодам авую за пърисылко
__main__:18 - INFO - 0_14_10 ref: iз гистроу въ 12 д декабря 1712
__main__:19 - INFO - 0_14_10 hyp: iз гистроу въ 12 д декабря 1712
__main__:18 - INFO - 0_18_13 ref: заполочным до элбинга а оттол отпусти iх з бата
__main__:19 - INFO - 0_18_13 hyp: за полочным до элбинга а оттол отпусти iх з бата
__main__:18 - INFO - 0_18_12 ref: празником (+ також возми с собою i афицероф
__main__:19 - INFO - 0_18_12 hyp: празником i також возми собою i афицероф
__main__:18 - INFO - 0_18_11 ref: p s поздравъляю вам сим торжественным
__main__:19 - INFO - 0_18_11 hyp: p s поздравляю вам сим торжественным
__m

5.513829678327926

In [93]:
val_model_with_decoder(test_loader, train_geval17_lm_decoder)

  0%|          | 0/483 [00:00<?, ?it/s]

trainscript:42 - INFO - loss: 5.51383
__main__:18 - INFO - 0_14_12 ref: съ юшковым поклонис от меня кнеiме да
__main__:19 - INFO - 0_14_12 hyp: съ юшковым поклонис от меня к не iмеда
__main__:18 - INFO - 0_14_13 ref: ниловичевой
__main__:19 - INFO - 0_14_13 hyp: нилови вой
__main__:18 - INFO - 0_14_11 ref: p s благодарствую за пърисылку
__main__:19 - INFO - 0_14_11 hyp: p s благода авую за пърисылко
__main__:18 - INFO - 0_14_10 ref: iз гистроу въ 12 д декабря 1712
__main__:19 - INFO - 0_14_10 hyp: iз гистроу въ 12 д декабря 1712
__main__:18 - INFO - 0_18_13 ref: заполочным до элбинга а оттол отпусти iх з бата
__main__:19 - INFO - 0_18_13 hyp: за полочным до элбинга а оттол отпусти iх з бата
__main__:18 - INFO - 0_18_12 ref: празником (+ також возми с собою i афицероф
__main__:19 - INFO - 0_18_12 hyp: празником i також возми собою i афицероф
__main__:18 - INFO - 0_18_11 ref: p s поздравъляю вам сим торжественным
__main__:19 - INFO - 0_18_11 hyp: p s поздравляю вам сим торжественным
__ma

5.513829678327926